In [42]:
# --- CELL 1: SETUP ---
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import scipy.optimize
import scipy.ndimage
import ipywidgets as widgets
from IPython.display import display, clear_output
import seaborn as sns

# Set Style
sns.set_theme(style="whitegrid", context="notebook", font_scale=1.1)

print("Libraries loaded. Proceed to Cell 2.")

Libraries loaded. Proceed to Cell 2.


In [43]:
# --- CELL 2: HYBRID LOGIC ENGINE ---

def find_bifurcation_points_numerical(f_lam, df_lam, a_bounds, x_bounds):
    """
    Fallback: Robustly finds bifurcation points by scanning the grid
    for local minima of f^2 + df^2.
    """
    res_scan = 60
    a_scan = np.linspace(a_bounds[0], a_bounds[1], res_scan)
    x_scan = np.linspace(x_bounds[0], x_bounds[1], res_scan)
    A_scan, X_scan = np.meshgrid(a_scan, x_scan)

    val_f = f_lam(A_scan, X_scan)
    val_df = df_lam(A_scan, X_scan)
    error_metric = np.log10(val_f**2 + val_df**2 + 1e-15)

    local_min = scipy.ndimage.minimum_filter(error_metric, size=3) == error_metric
    candidates = (error_metric < -2) & local_min

    bif_points = []
    indices = np.argwhere(candidates)

    for idx in indices:
        row, col = idx
        a_guess, x_guess = A_scan[row, col], X_scan[row, col]

        def objective(vars):
            a, x = vars
            return f_lam(a, x)**2 + df_lam(a, x)**2

        res = scipy.optimize.minimize(objective, [a_guess, x_guess], bounds=[a_bounds, x_bounds], method='L-BFGS-B')

        if res.fun < 1e-5:
            sol = res.x
            is_new = True
            for seen in bif_points:
                if np.linalg.norm(sol - np.array(seen)) < 0.05 * (a_bounds[1] - a_bounds[0]):
                    is_new = False
                    break
            if is_new:
                bif_points.append(sol)

    return bif_points

def format_coord(val):
    """Smart formatting: 1.0 -> '1', 0.57735 -> '0.577'"""
    if abs(val - round(val)) < 1e-9:
        return f"{int(round(val))}"
    return f"{val:.4g}"

def get_bifurcation_plot(eqn_str, param_name, var_name, a_min, a_max, x_min, x_max):
    fig, ax = plt.subplots(figsize=(10, 6))

    try:
        # --- 1. PARSE & SANITIZE ---
        eqn_str = eqn_str.replace('^', '**')
        expr = sp.sympify(eqn_str)

        user_param = sp.Symbol(param_name)
        user_var = sp.Symbol(var_name)

        # Auto-detect constants (set to 1.0)
        all_syms = expr.free_symbols
        subs_dict = {}
        undefined_vars = []
        for s in all_syms:
            if s != user_param and s != user_var:
                subs_dict[s] = 1.0
                undefined_vars.append(str(s))

        expr_clean = expr.subs(subs_dict)
        deriv_expr = sp.diff(expr_clean, user_var)

        # --- 2. SOLVE BIFURCATION POINTS (HYBRID) ---
        bif_pts = []
        method_used = "Numerical"

        # Attempt A: Analytical (Symbolic)
        try:
            # Solve system {f=0, df/dx=0}
            sols = sp.solve([expr_clean, deriv_expr], [user_param, user_var], dict=True)

            for s in sols:
                # Convert to complex to check for imaginary parts
                val_a = complex(s[user_param].evalf())
                val_x = complex(s[user_var].evalf())

                # Keep only Real results
                if abs(val_a.imag) < 1e-6 and abs(val_x.imag) < 1e-6:
                    ra, rx = val_a.real, val_x.real
                    if (a_min <= ra <= a_max) and (x_min <= rx <= x_max):
                        bif_pts.append((ra, rx))

            if len(bif_pts) > 0:
                method_used = "Analytical"
        except Exception:
            pass

        # Attempt B: Numerical (Fallback)
        if len(bif_pts) == 0:
            f_lam_num = sp.lambdify((user_param, user_var), expr_clean, modules=['numpy', 'scipy'])
            df_lam_num = sp.lambdify((user_param, user_var), deriv_expr, modules=['numpy', 'scipy'])
            bif_pts = find_bifurcation_points_numerical(f_lam_num, df_lam_num, (a_min, a_max), (x_min, x_max))
            method_used = "Numerical"

        # --- 3. NUMERICAL WRAPPERS FOR PLOTTING ---
        f_lam = sp.lambdify((user_param, user_var), expr_clean, modules=['numpy', 'scipy'])
        df_lam = sp.lambdify((user_param, user_var), deriv_expr, modules=['numpy', 'scipy'])

        # --- 4. GRID GENERATION ---
        res = 300
        if a_min >= a_max: a_max = a_min + 1.0
        if x_min >= x_max: x_max = x_min + 1.0

        a_lin = np.linspace(a_min, a_max, res)
        x_lin = np.linspace(x_min, x_max, res)
        A, X = np.meshgrid(a_lin, x_lin)

        Z = f_lam(A, X)
        dZ = df_lam(A, X)

        Z = np.nan_to_num(Z, nan=0.0)
        dZ = np.nan_to_num(dZ, nan=0.0)

        # --- 5. PLOT REGIONS ---
        from matplotlib.colors import ListedColormap
        cmap_regions = ListedColormap(['#ffe6e6', '#e6f2ff'])
        ax.pcolormesh(A, X, np.sign(Z), cmap=cmap_regions, alpha=0.5, shading='auto', zorder=0)

        # --- 6. PLOT ARROWS ---
        arrow_res = 18
        aa = np.linspace(a_min, a_max, arrow_res)
        xx = np.linspace(x_min, x_max, arrow_res)
        AA, XX = np.meshgrid(aa, xx)
        VV = f_lam(AA, XX)

        threshold = 0.05 * np.max(np.abs(Z)) if np.max(np.abs(Z)) > 0 else 1e-6
        mask_sig = np.abs(VV) > threshold

        U = np.zeros_like(VV)
        V_norm = np.sign(VV)

        mask_up = (VV > 0) & mask_sig
        ax.quiver(AA[mask_up], XX[mask_up], U[mask_up], V_norm[mask_up],
                  color='#004488', alpha=0.5, width=0.003, scale=40, headwidth=4, zorder=2)

        mask_down = (VV < 0) & mask_sig
        ax.quiver(AA[mask_down], XX[mask_down], U[mask_down], V_norm[mask_down],
                  color='#880000', alpha=0.5, width=0.003, scale=40, headwidth=4, zorder=2)

        # --- 7. PLOT LINES ---
        Z_stable = np.ma.masked_where(dZ >= 0, Z)
        ax.contour(A, X, Z_stable, levels=[0], colors='black', linewidths=2.5, linestyles='solid', zorder=3)

        Z_unstable = np.ma.masked_where(dZ <= 0, Z)
        ax.contour(A, X, Z_unstable, levels=[0], colors='black', linewidths=2.5, linestyles='dashed', zorder=3)

        # --- 8. PLOT & LABEL POINTS ---
        for (bp_a, bp_x) in bif_pts:
            ax.plot(bp_a, bp_x, 'o', color='#ff00ff', markersize=8, markeredgecolor='black', zorder=4)
            label = f"({format_coord(bp_a)}, {format_coord(bp_x)})"
            ax.annotate(label, (bp_a, bp_x), xytext=(5, 5), textcoords='offset points',
                        fontsize=10, fontweight='bold', bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8))

        # --- 9. FORMATTING ---
        title_extra = ""
        if undefined_vars:
            title_extra = f"\n(Constants set to 1.0: {', '.join(undefined_vars)})"

        ax.set_title(f"d{var_name}/dt = {sp.latex(expr)}{title_extra}", fontsize=14)
        ax.set_xlabel(f"Parameter ({param_name})", fontsize=12)
        ax.set_ylabel(f"Variable ({var_name})", fontsize=12)
        ax.set_xlim(a_min, a_max)
        ax.set_ylim(x_min, x_max)

        from matplotlib.patches import Patch
        from matplotlib.lines import Line2D
        legend_elements = [
            Line2D([0], [0], color='black', lw=2.5, linestyle='-', label='Stable'),
            Line2D([0], [0], color='black', lw=2.5, linestyle='--', label='Unstable'),
            Line2D([0], [0], color='#ff00ff', marker='o', lw=0, label=f'Bifurcation ({method_used})'),
            Patch(facecolor='#e6f2ff', edgecolor='none', alpha=0.6, label='Flow Up'),
            Patch(facecolor='#ffe6e6', edgecolor='none', alpha=0.6, label='Flow Down'),
        ]
        ax.legend(handles=legend_elements, loc='upper right', frameon=True, framealpha=0.9)

    except Exception as e:
        ax.text(0.5, 0.5, f"Error:\n{str(e)}", ha='center', va='center', color='red', transform=ax.transAxes)
        ax.set_xlim(a_min, a_max)
        ax.set_ylim(x_min, x_max)

    plt.tight_layout()
    return fig

print("Hybrid Logic Engine Defined.")

Hybrid Logic Engine Defined.


In [44]:
# --- CELL 3: CONFIGURATION ---

# 1. Default Start Values
DEFAULT_CONFIG = {
    'eqn': 'x**3 - x + a',
    'param': 'a',
    'var': 'x',
    'amin': -2.0, 'amax': 2.0,
    'xmin': -2.0, 'xmax': 2.0
}

# 2. Preset Dictionary
PRESETS = {
    "Saddle-Node (Hysteresis)": "x**3 - x + a",
    "Pitchfork (Supercritical)": "x**3 - a*x",
    "Transcritical": "x**2 - a*x",
    "Simple Saddle-Node": "x**2 - a",
    "Logistic Growth": "r*x*(1 - x/k)",
    "Sine Function": "sin(x) + a",
    "Exponential": "a - x - exp(-x)"
}

print("✅ Configuration Loaded.")

✅ Configuration Loaded.


In [45]:
# --- CELL 4: INTERACTIVE DASHBOARD ---

style = {'description_width': 'initial'}

# 1. TOP ROW: Equation Input
lbl_d1 = widgets.HTML("<b>d</b>")
w_var_in = widgets.Text(value=DEFAULT_CONFIG['var'], layout=widgets.Layout(width='60px'))
lbl_d2 = widgets.HTML("<b>&nbsp;/&nbsp; dt</b>")
lbl_eq = widgets.HTML("<b>&nbsp;=&nbsp;</b>")
w_eqn = widgets.Text(value=DEFAULT_CONFIG['eqn'], placeholder='Equation', layout=widgets.Layout(width='400px'))

btn_plot = widgets.Button(description="Plot", button_style='primary', icon='play', layout=widgets.Layout(width='100px'))

top_row = widgets.HBox([
    lbl_d1, w_var_in, lbl_d2, lbl_eq, w_eqn, btn_plot
], layout=widgets.Layout(justify_content='center', align_items='center', margin='0 0 15px 0'))


# 2. OUTPUT AREA
plot_output = widgets.Output()


# 3. BOTTOM ROW: Configuration
w_param_in = widgets.Text(value=DEFAULT_CONFIG['param'], description='<b>Param:</b>', style=style, layout=widgets.Layout(width='150px'))

# Domain Controls
lbl_dom = widgets.HTML("<b>Domain (Param):</b>")
w_amin = widgets.FloatText(value=DEFAULT_CONFIG['amin'], description='Min:', style=style, layout=widgets.Layout(width='100px'))
w_amax = widgets.FloatText(value=DEFAULT_CONFIG['amax'], description='Max:', style=style, layout=widgets.Layout(width='100px'))

# Range Controls
lbl_rng = widgets.HTML("<b>Range (Var):</b>")
w_xmin = widgets.FloatText(value=DEFAULT_CONFIG['xmin'], description='Min:', style=style, layout=widgets.Layout(width='100px'))
w_xmax = widgets.FloatText(value=DEFAULT_CONFIG['xmax'], description='Max:', style=style, layout=widgets.Layout(width='100px'))

config_grid = widgets.VBox([
    widgets.HBox([w_param_in]),
    widgets.HBox([lbl_dom, w_amin, w_amax]),
    widgets.HBox([lbl_rng, w_xmin, w_xmax])
], layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='0 10px 0 0', width='50%'))


# 4. PRESETS
w_preset = widgets.Dropdown(options=[(k, v) for k, v in PRESETS.items()], description='<b>Preset:</b>', style=style)
btn_load = widgets.Button(description="Load", icon="arrow-up")

dict_html = widgets.HTML("""
<style>
    .dict-entry { margin-bottom: 2px; font-family: monospace; font-size: 0.9em; color: #444; }
    .dict-key { color: #d63384; font-weight: bold; }
</style>
<div style="margin-top: 5px;">
    <div class="dict-entry"><span class="dict-key">Power:</span> x**2, x**3</div>
    <div class="dict-entry"><span class="dict-key">Trig:</span> sin(x), tan(x)</div>
    <div class="dict-entry"><span class="dict-key">Exp:</span> exp(x), log(x)</div>
    <div class="dict-entry"><span class="dict-key">Const:</span> k=1.0 (auto)</div>
</div>
""")

preset_panel = widgets.VBox([
    widgets.HTML("<b>📂 Tools</b>"),
    widgets.HBox([w_preset, btn_load]),
    dict_html
], layout=widgets.Layout(border='1px solid #ddd', padding='10px', width='50%'))


# --- HANDLERS ---
def on_plot_click(_):
    plot_output.clear_output(wait=True)
    with plot_output:
        fig = get_bifurcation_plot(
            w_eqn.value,
            w_param_in.value,
            w_var_in.value,
            w_amin.value, w_amax.value,
            w_xmin.value, w_xmax.value
        )
        plt.show(fig)

def on_load_click(_):
    w_eqn.value = w_preset.value
    if "Logistic" in w_preset.label:
        w_param_in.value = 'r'
        w_var_in.value = 'x'
        w_amin.value, w_amax.value = -1, 4
        w_xmin.value, w_xmax.value = -1, 2
    elif "Pitchfork" in w_preset.label:
        w_param_in.value = 'a'
        w_var_in.value = 'x'
        w_amin.value, w_amax.value = -4, 4
        w_xmin.value, w_xmax.value = -4, 4
    elif "Transcritical" in w_preset.label:
        w_param_in.value = 'a'
        w_var_in.value = 'x'
        w_amin.value, w_amax.value = -4, 4
        w_xmin.value, w_xmax.value = -4, 4

    on_plot_click(None)

btn_plot.on_click(on_plot_click)
btn_load.on_click(on_load_click)
w_eqn.observe(on_plot_click, names='value')

# --- DISPLAY ---
bottom_row = widgets.HBox([config_grid, preset_panel])
ui = widgets.VBox([top_row, plot_output, bottom_row])

on_plot_click(None)
display(ui)

In [46]:
# --- CELL 5: UNIT TESTS ---
def run_tests():
    print("Running Diagnostics...")

    test_cases = [
        # (Equation, Param, Var, Expected_Points_Count)
        ("x**3 - a*x", "a", "x", 1), # Pitchfork (1 split point at 0,0)
        ("x**2 - a", "a", "x", 1),   # Saddle-node (1 point at 0,0)
        ("x**2 - a*x", "a", "x", 1), # Transcritical (1 point at 0,0)
    ]

    for eqn, p, v, expected_pts in test_cases:
        print(f"  > Testing: {eqn}...", end=" ")
        try:
            # We use a mocked plotting call to check logic without rendering
            # Re-using the parsing logic from Cell 2
            eqn_str = eqn.replace('^', '**')
            expr = sp.sympify(eqn_str)
            user_param = sp.Symbol(p)
            user_var = sp.Symbol(v)
            deriv = sp.diff(expr, user_var)

            # Analytical Check
            sols = sp.solve([expr, deriv], [user_param, user_var], dict=True)
            valid_pts = [s for s in sols if abs(complex(s[user_param]).imag) < 1e-6]

            if len(valid_pts) >= expected_pts:
                print(f"PASS (Found {len(valid_pts)} bifurcation points)")
            else:
                print(f"WARN (Expected {expected_pts}, found {len(valid_pts)})")

        except Exception as e:
            print(f"FAIL ({str(e)})")

    print("\nDiagnostics Complete.")

run_tests()

Running Diagnostics...
  > Testing: x**3 - a*x... PASS (Found 1 bifurcation points)
  > Testing: x**2 - a... PASS (Found 1 bifurcation points)
  > Testing: x**2 - a*x... PASS (Found 1 bifurcation points)

Diagnostics Complete.
